
# TODO

~~read test files~~

do normalization separately for fbank and mfcc?

~~padding~~

**Use 48 for training and map after prediction**

outliers?

adam?

use custom metric (edit distance)

~~BLSTM~~

monitor for modelcheckpoint and earlystopping

~~no validation?~~

**Add padding as 40th class?**

dropout / recurrent dropout

In [144]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, Bidirectional, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import arrow

# GPU usage
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
set_session(tf.Session(config=config))

In [201]:
# np.random.seed(666)
NUM_UNITS = 100
DROPOUT = 0.1
# RECURRENT_DROPOUT = 0.3
FEATURE_LEN = 108
# NUM_EPOCHS = 3
BATCH_SIZE = 64
VERBOSE = 2
OPTIMIZER = 'rmsprop'
NUM_CLASSES = 39
DATA_PATH = '../data/'
MODEL_PATH = '../models/'
PREDICTION_PATH = '../predictions/'

In [145]:
def read_ark(filepath, sentence2frames):
    with open(filepath) as train_ark_f:
        last_sentence = ''
        frames = list()
        is_first_line = True
        for line in train_ark_f:
            line = line.strip().split()
            instance_id = line[0]
            speaker_id, sentence_id, frame_id = instance_id.split('_')
            sentence = '_'.join((speaker_id, sentence_id))
            features = list(map(float, line[1:]))

            if sentence != last_sentence and not is_first_line:
                if last_sentence not in sentence2frames:
                    sentence2frames[last_sentence] = frames
                else:
                    sentence2frames[last_sentence] = np.concatenate(
                        (sentence2frames[last_sentence], frames),
                        axis=1
                    )
                frames = list()

            last_sentence = sentence
            frames.append(features)
            is_first_line = False

        # process last sentence
        if last_sentence not in sentence2frames:
            sentence2frames[last_sentence] = frames
        else:
            sentence2frames[last_sentence] = np.concatenate(
                (sentence2frames[last_sentence], frames),
                axis=1
            )
        frames = list()

In [146]:
%%time

sentence2frames = dict()
read_ark(DATA_PATH + 'fbank/train.ark', sentence2frames)
read_ark(DATA_PATH + 'mfcc/train.ark', sentence2frames)

CPU times: user 53 s, sys: 2.87 s, total: 55.9 s
Wall time: 1min 26s


In [147]:
# number of training sentences
NUM_SENTENCES = len(sentence2frames)
print(NUM_SENTENCES)

# number of frames in this sentence
print(len(sentence2frames['faem0_si1392']))

# number of fbank+mfcc features in a frame
print(len(sentence2frames['faem0_si1392'][0]))

# first feature
print(sentence2frames['faem0_si1392'][0][0])

3696
474
108
3.148541


In [148]:
MAX_SENTENCE_LEN = 0
for s in sentence2frames:
    MAX_SENTENCE_LEN = max(MAX_SENTENCE_LEN, len(sentence2frames[s]))
print(MAX_SENTENCE_LEN)

777


In [172]:
phone48_phone39 = dict()
phone39_int39 = dict()
int39_phone39 = dict()
phone48_char48 = dict()

with open(DATA_PATH + 'phones/48_39.map') as phone48_phone39_f:
    for line in phone48_phone39_f:
        phone48, phone39 = line.strip().split('\t')
        phone48_phone39[phone48] = phone39
        if phone39 not in phone39_int39:
            int39 = len(phone39_int39)
            phone39_int39[phone39] = int39
            int39_phone39[int39] = phone39

with open(DATA_PATH + '48phone_char.map') as phone48_char48_f:
    for line in phone48_char48_f:
        phone48, int48, char48 = line.strip().split('\t')
        phone48_char48[phone48] = char48

In [150]:
%%time

def read_label(filepath, sentence2labels):
    with open(filepath) as train_lab_f:
        last_sentence = ''
        labels = list()
        is_first_line = True
        
        for line in train_lab_f:
            instance_id, label = line.strip().split(',')
            speaker_id, sentence_id, frame_id = instance_id.split('_')
            sentence = '_'.join((speaker_id, sentence_id))
            index = phone39_int39[phone48_phone39[label]]

            if sentence != last_sentence and not is_first_line:
                sentence2labels[last_sentence] = labels
                labels = list()

            last_sentence = sentence
            labels.append(index)
            is_first_line = False

        # process last sentence
        sentence2labels[last_sentence] = labels
        labels = list()

sentence2labels = dict()
read_label(DATA_PATH + 'label/train.lab', sentence2labels)

CPU times: user 1.44 s, sys: 16 ms, total: 1.46 s
Wall time: 1.79 s


In [151]:
# number of training sentences
print(len(sentence2labels))

# number of frames in this sentence
print(len(sentence2labels['faem0_si1392']))

# first label
print(sentence2labels['faem0_si1392'][0])

3696
474
7


In [67]:
%%time

def one_hot(n, i):
    ans = np.zeros(n)
    ans[i] = 1
    return ans

sentences = sorted(sentence2labels)
X = np.zeros((NUM_SENTENCES, MAX_SENTENCE_LEN, FEATURE_LEN))
y = np.zeros((NUM_SENTENCES, MAX_SENTENCE_LEN, NUM_CLASSES))

for i, s in enumerate(sentences):
    frames = sentence2frames[s]
    X[i, :len(frames)] = frames
    y[i, :len(frames)] = np.stack([one_hot(NUM_CLASSES, l) for l in sentence2labels[s]])

CPU times: user 4.48 s, sys: 1.41 s, total: 5.88 s
Wall time: 6.65 s


In [68]:
print(X.shape)
print(y.shape)

(3696, 777, 108)
(3696, 777, 39)


In [69]:
def build_model(verbose=True):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=NUM_UNITS, return_sequences=True), input_shape=(MAX_SENTENCE_LEN, FEATURE_LEN)))
    model.add(Dropout(DROPOUT))
    model.add(TimeDistributed(Dense(NUM_CLASSES, activation='softmax')))
    model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

model = build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 777, 200)          167200    
_________________________________________________________________
dropout_6 (Dropout)          (None, 777, 200)          0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 777, 39)           7839      
Total params: 175,039
Trainable params: 175,039
Non-trainable params: 0
_________________________________________________________________


In [70]:
monitor = 'loss'
model_filepath = MODEL_PATH + model_{monitor}'.format(monitor=monitor) + '_{epoch:03d}_{loss:.4f}.h5'
checkpoint = ModelCheckpoint(model_filepath, monitor=monitor, verbose=1, save_best_only=True)
earlystop = EarlyStopping(monitor=monitor, patience=2, verbose=0)

model.fit(X, y, batch_size=BATCH_SIZE, epochs=500, verbose=1, callbacks=[earlystop, checkpoint])

Epoch 1/500
 320/3696 [=>............................] - ETA: 185s - loss: 1.2607 - acc: 0.1746

KeyboardInterrupt: 

In [ ]:
%%time

sentence2frames_test = dict()
read_ark(DATA_PATH + 'fbank/test.ark', sentence2frames_test)
read_ark(DATA_PATH + 'mfcc/test.ark', sentence2frames_test)

In [100]:
# number of testing sentences
print(len(sentence2frames_test))

592
CPU times: user 8.34 s, sys: 212 ms, total: 8.55 s
Wall time: 10.2 s


In [101]:
%%time

def one_hot(n, i):
    ans = np.zeros(n)
    ans[i] = 1
    return ans

NUM_SENTENCES_TEST = 592
sentences_test = sorted(sentence2frames_test)
X_test = np.zeros((NUM_SENTENCES_TEST, MAX_SENTENCE_LEN, FEATURE_LEN))

for i, s in enumerate(sentences_test):
    frames = sentence2frames_test[s]
    X_test[i, :len(frames)] = frames

print(X_test.shape)

(592, 777, 108)
CPU times: user 24 ms, sys: 24 ms, total: 48 ms
Wall time: 47 ms


In [197]:
%%time

model_name = 'model_val_loss_030_0.2851.h5'
model = keras.models.load_model(MODEL_PATH + model_name)
y_test_onehot = model.predict(X_test)
y_test = np.argmax(y_test_onehot, axis=-1)

print(y_test_onehot.shape)
print(y_test.shape)

(592, 777, 39)
(592, 777)
CPU times: user 14 s, sys: 568 ms, total: 14.6 s
Wall time: 14.9 s


In [198]:
def decode(seq):
    L = len(seq)
    sil = phone39_int39['sil']
    
    # trim leading sil
    for i in range(L):
        if seq[i] != sil:
            break
    start = i

    # trim trailing sil
    is_sil = False
    for i in range(L - 1, -1, -1):
        if is_sil:
            if seq[i] != sil:
                break
        else:
            if seq[i] == sil:
                is_sil = True
    end = i + 1
    
    trimmed = [phone48_char48[int39_phone39[i]] for i in seq[start:end]]
    return''.join([k for k, v in itertools.groupby(trimmed)])

y_predicted = list(map(decode, y_test))

time = arrow.now('Asia/Taipei').format('YYYYMMDD_HHmmss')
prediction_filepath = PREDICTION_PATH + prediction_{time}.csv'.format(time=time)
with open(prediction_filepath, 'w') as prediction_f:
    prediction_f.write('id,phone_sequence\n')
    for i, sentence in enumerate(sentences_test):
        prediction_f.write('{id},{y}\n'.format(id=sentence, y=y_predicted[i]))

In [ ]:
monitor = val_acc
Epoch 36/200
3326/3326 [==============================] - 77s - loss: 0.3140 - acc: 0.2903 - val_loss: 0.3337 - val_acc: 0.2845

monitor = val_loss
Epoch 23/500
3326/3326 [==============================] - 172s - loss: 0.3495 - acc: 0.2870 - val_loss: 0.3622 - val_acc: 0.2925

monitor = acc (no validation)
Epoch 133/500
3696/3696 [==============================] - 109s - loss: 0.2369 - acc: 0.3140